# **Designing Reagents for your CRISPR experiment**

For this project, you  will use the concepts you've learned over the past two weeks to develop a program that will generate the sequences of the nucleic acid reagents necessary to carry out a particular CRISPR modification. Specifically, you will be generating the sequence of a **crRNA** and **repair template** that will add a specific tag to some gene of interest.

Your program will take 3 input parameters to generate these two reagents:
1. The DNA sequence of the gene you are modifying (including UTRs and introns) *in wormbase format* (meaning non-coding bases should be lower-case, and coding bases will be upper-case). We'll call this the target gene.
2. The DNA sequence of the tag you wish to add. 
3. The terminus of the target protein you wish to add the tag to (either `"N"` or `"C"`)

This is a fairly involved project, so we will be splitting it into two parts. During this first week working, you should accomplish the following steps:
1. Set up your input parameters (target sequence, tag sequence, and terminus)
2. Find the position of the insert in the gene sequence
3. Find the closest PAM to the insert position and determine which strand it is on.
4. Determine the sequence of the crRNA (this *will* depend on which strand that your PAM is on).

We will end our Python unit at that point and design the repair template we need to edit unc-32.  However, we will include material so that you can design a repair template in python for those students who are curious and have the time and energy.  Again, this is completely optional and will not be covered this semester. 
During the second week of the assignment, you should accomplish the following steps:
1. Determine if you can modify the PAM sequence without disrupting the coding sequence of your target gene.
2. Design the repair template such that the PAM is modified if possible, and the crRNA-binding region is modified if not possible.
  
  - The final repair template should have 60bp homology arms on either side of any modifications, and should include the tag sequence.<br>

It's likely that this project will be quite hard for some of you, but you will have your TAs to help you through the concepts you're struggling with. One general tip many of you may have already induced is that going over the logic of steps before diving into syntax is helpful, a process called pseudocoding.  We recommend that before you worry about writing the code and syntax, you first "pseudocode".  Pseudocode refers to non-coding language used to explicitly describe algorithms, so that one can focus on the logic of the algorithm without being distracted by details of syntax. For example, last week you were asked to find the number of amino acids that a DNA sequence encodes.  If you were solving this in psuedocode you might write the following steps:
1) determine which nucleotides in a DNA sequence are coding nucleotides
2) find the total number of coding nucleotides
3) to obtain the toal number of amino acids, divide total number of nucleotides by 3.

At the end of the project, **you will have a tool that you can use to generate real CRISPR reagents**, with some important caveats that we will note at the end of the assignment.

It is also import to note that **google is your friend**. If your code is throwing an error and you can't figure out why, someone else has almost certainly had the same issue before and you should be able to find an answer on a site like Stack Overflow. Googling is not cheating. Just don't copy someone else's CRISPR reagent design program if you find one.

With that being said, let's get started!

---
# Set your parameters

You have a code cell below that you'll use to assign variables to your three input parameters. As a reminder, you should have three parameters:
1. The DNA sequence of your target gene
2. The DNA sequence of your tag
3. The terminus at which the tag will be inserted in the target protein (either `"N"` or `"C"`).

The target gene sequence should contain both 5' UTR and 3' UTR regions, and should be in wormbase format, such that non-coding segments are lowercase and coding segments are upper case.

A user should be able to modify these three variables with the parameters they want. For now:
1. For our target sequence, we're using a slightly modified version of the *unc-32* gene [here](https://wormbase.org/species/c_elegans/transcript/ZK637.8a.1)
  * The amino acid sequence is the same, we've just modified the gene slightly so there's less ambiguity about which PAM to choose
2. We're using GFP  as our tag
  * Currently we have a placeholder, but we'll give you the full GFP sequence we're using before next week
3. We'll insert the tag at the C-terminus of Unc-32

In [ ]:
target_seq = 'ggaacttggagacatgacaacaatacaattgttttaaaacttacaatatattccaaagcacaactggaagattcataatcaagagctgcctttgcgagcattgttagacttgagtttcgggtcaatgagttcactgaaggagaatctctacctccattcaggcgtttcgcgggagatctgtaaaatcataagattagtcggccactttttcggaacagcttgttactttttgggagatccggtacgtttgatactgcgagacggtgataggtatagttcttcattacgatatctcgatggaactttagatgtttcacggagactgtatggtgagctggtgcgatcctgaaaataccgaatgtcatgcatactttcaatatctttctttaatttacaacttctttatttttaaaaacacgaaaacccaataaaatcaatatttccaataacttacaggatcagacgtgtcacttgctgcctttttccgcggacttctcaccgcagacgacatttttatctgaaaaataacaaatttttgaataatttcagaccacgagacaaaaatcaatgaaagaatgcggacgcgcgcgcgcgaaaaaaactttgaaatggcggttcttttccccaaccaacagccgatttcaaacagcgtgatggtctcgacgcgattgccctgcggcagtgtagcgaggtacggtggagcgctgtctgttggaaaacagaacacagctgagagaagtgaatgtggtgggaaacggaaaaaagggagagctgctgtttgacagtcgaacgtcgtgacattccgtttttggattttctcgtgaaagttttttttttttttttatttgttttatttcttatttgtattgattttccatctttcctgggttatcattaaattttaaacatggttttacacaagatcatgcggtgaaacatgtttttttcttcctttccgcatttaaaacgaattattttgtttatattgttcttctctttccattctcatgattcatttctctccttattggcaagaaacattataaaattgttcattcctactaagtgtttaccaaaatttattttggccgcaaagttttaaaattgtagaataaacttttcattaaaaatatttcgtcttcaatcactcatctttgttccgaaaacatctcacaggttttttcttcaaagcaatattcttctttagtgtttttagtcagcgtttttcgttctggatctggattctatggtttgatgttactttgaaacactatttataaaatctttttgcaaagaaacaattctcttaaatttaatacaatttccgcatcaaaattgaagtttttgtgcaaagttgacctctcgcagcgtaatcttatcgttttttcgcgttatcaccactcaatctcccccactgctctacaattatttcatttctctcaaccttttgatcgaatcgattgaatttcataatatccacgaaatgtgagtcgcccaatgtttacgtttctccatcctataacactgcttacaaatggtttcagtgttcaaacattgcggtgaatgaggcagacaatcagacattttctttcttttaaaacctgtcctgttcatttcttcctcatctcctttaggttttcttattcgccatctgaaagtttatttctctgcgtctagctattagatcctggagaattacgctctaatgtcgatgacgtgtcggctgatcgattagattgcgcaaattgtttctttagttttttctagatttctctcccttttttcatgatattccaatggagcacgtttgagcgcatgctctcttcttgctcagtgctgctccgtatcgatccctctccgtcaatatctggtcttcggtcctgaggccttcgctcctagcctcgtgcttttcttaaatgttttctctcgaaagaagcgtttttgatttttttttccttcttatttttgaaatcttcatattatttcatgcaaattcttacagaatcaactagaATGGGTGATTACGTGACTCCCGGCGAGGAGCCACCACAACCGGGCATCTATCGAAGTGAGCAGATGTGCCTGGCTCAACTCTACCTTCAATCTGATGCTTCCTATCAATGCGTTGCTGAACTGGGAGAGCTCGGACTTGTTCAGTTTCGTGATgtgagtttgtgatatgtaaaaaactattttcaattaataattttaaaaaaaagtaaaacttcaatttattttaattttcagCTTAATCCCGATGTGAGCTCCTTCCAACGAAAATATGTAAATGAAGTCAGACGGTGTGATGAAATGGAACGAAAATTGAGATATCTTGAACGAGAAATTAAAAAAGATCAGATCCCGATGCTGGATACTGGTGAGAATCCAGATGCTCCACTTCCACGAGAAATGATTGATTTGGAGgtttgttgaaaaaaaaatataggtatataaatatgtgttgcaatttcacggtctattttttttgaatgtttaatttttaaaaagtttttaaaaattctttcttacagGCAACATTCGAAAAACTCGAGAACGAACTTCGCGAGGTCAATAAGAACGAGGAAACGCTGAAGAAAAACTTTTCAGAGCTCACAGAGCTGAAACATATTTTACGAAAGACTCAAACTTTCTTCGAAGAGgtgagaatttcatgttttctcatcatttccaaattctctaacattttttttcacaaattctccttttctctctaattctggcatgcttttcgagaatagttaaacaccctaacataatcaaaaaaaaaatggaacttgatcctataatattttttattttctagcagatgtagtatgttttgtgcctcacctatttatagctataatattagtatagttgctctcctcactattcggttcgtcacacagGTTGATCATGATCGGTGGCGAATTCTGGAAGGCGGAAGTGGACGACGAGGACGTTCTACAGAACGTGAAGAAACGCGACCCCTTATTGATATTGGAGATATGGACGACGATTCAGCTGCACGGATGTCAGCTCAAGCTGCGATGCTACGTCTTGGgtatgtggtcctaggcaagatggacagaccagaaagcgccaccatcgcgaaacgagacctagtttatgttgtcttgttcgtatccttctccttttgcatcccgttggtgttttttcctgattcttttctggtaatttaattaattggtgcttttaagcactaggaaaatttaaaatgactcctaatctaatatcttcccacatctttctctctccttaaatcctcccctaaccgactttccttatccttctatgttccttttccgtaaaatctccttcactaacacaggccgggactggagaaatgttgccacctgctgcagtcgaatctgaagaaggccttgaactgactcaacatgccgccgctggcggagccacaatgttcgccaattttgggtgagacttctgctcctactcgtcgttgtatcgttcattgtcgcacggttttgctttcgattgcttacacttctttccttttctatcagtatctcatgtctctatcctgtagcttgtcgtggtgtgccagagtttaggtgaatcggtcactacatccccaacataactttggaaagaatttattgaaaaaagccaagggtaatctaaatgtagtattgtgtttgtttgattctgcagcacgaagacatgattgcctcatcagcggaaagttcgggaattggtgaagtgctcagtgccgacgaagaagagctttcaggaagattcagcgatgcaatgtcgccactcaaactgcaattacggtaggatcaggcttattttgttgtctttttgtcttttcatatcattatgtattgtgatggtggtgtcttttcaaagcgagcgcgttaaaagatgtgtccggcttggtttcttagtttttagaacatgaaaatcatcatttacttaaatttttgattttagATTTGTTGCTGGTGTAATTCAACGGGAACGTCTTCCCGCATTTGAGCGACTTCTTTGGAGAGCGTGTCGTGGTAATGTCTTCTTGCGAACAAGCGAGATTGATGATGTACTCAATGATACGGTCACTGGAGATCCAGTCAACAAGTGCGTCTTCATCATCTTCTTCCAAGGAGATCATCTTAAAACGAAAGTTAAGAAAATTTGTGAAGGgtgagtttccgtaatttctaaaatcagaattttattcaaaacataatttttcagATTCCGCGCAACGCTTTACCCCTGTCCTGATACTCCACAAGAAAGACGAGAAATGTCAATTGGTGTGATGACTCGTATTGAAGATCTCAAAACTGTTCTCGGACAGACACAGGATCATCGTCATCGTGTTCTTGTCGCTGCATCGAAGAATGTTCGAATGTGGCTCACAAAAGTACGGAAAATCAAGTCGATCTACCATACACTAAACCTTTTCAATATCGATGTTACACAAAAGTGCTTGATCGCCGAGGTTTGGTGTCCGATTGCTGAGCTTGATCGTATCAAGATGGCGCTGAAACGTGGAACAGATGAGAGTGGAAGTCAAGTTCCGTCAATTTTGAATCGAATGGAGACAAATGAAGCTCCTCCGACATACAATAAGACGAACAAGTTCACAAAAGGATTCCAAAACATTGTTGATGCATATGGAATTGCAACATATCGAGAAATAAATCCAGCTCCATACACAATGATCTCGTTCCCTTTCCTTTTTGCTGTGATGTTCGGTGATATGGGGCACGGAGCCATCATGTTACTTGCTGCTCTTTTCTTTATTCTCAAAGAGAAACAACTCGAAGCGGCACGAATCAAAGATGAGATCTTCCAAACATTCTTTGGAGGTCGTTATGTGATCTTTTTGATGGGAGCTTTCTCAATATACACTGGATTCATGTACAATGATGTCTTCTCGAAAAGTATCAACACATTTGGGTCATCATGGCAGAATACAATTCCTGAAAGTGTTATTGATTATTACCTGGACGACGAGAAACGATCAGAATCTCAGCTTATTCTTCCACCAGAGACAGCTTTTGATGGAAATCCGTATCCAATTGGAGTGGATCCAGTTTGGAATCTTGCCGAAGGAAACAAATTGTCATTCCTCAACTCGATGAAAATGAAAATGTCCGTATTATTCGGAATTGCTCAAATGACATTCGGAGTTCTCCTCTCATATCAAAATTTCATATATTTCAAATCTGATCTTGATATTAAGTACATGTTCATTCCACAAATGATATTCTTGTCATCGATATTCATTTATCTGTGCATCCAAATCCTTTCAAAATGGCTATTCTTCGGTGCTGTTGGTGGAACTGTTCTTGGCTACAAGTATCCTGGTTCGAATTGTGCTCCATCCCTTCTCATCGGTCTCATCAACATGTTCATGATGAAAAGTCGTAATGCTGGATTTGTGGATGACAGTGGTGAAACATATCCACAGTGTTATTTGAGCACTTGGTATCCTGGACAGgtaagcttaatcctccccatgtctttcaggtgtttggatgactgatgttgatgaaattgaagagaaacgatgtttgacatgacgatgaataaaaacaaaagcaataatttttctatttaagtcgttcttcgaaacaattttcgtcctggtagcgatcgcgtgcgttcccgttatgctattcggaaagccttacttcttgtggaaagaggaaaaagaacggcgcgaggggggccatagacaattggtgagctattataataagaaaattgtttaaattagcacgcacccgccttctactgtcccgttactttttgttgtgttgtattgtttttattttgtgagaagatcgactttttaaaaataattttggagaacattttgcttcattcaaaattttaattttcacgaaagttttgaatcgcaaaggccatcaacatctgaaaatgctcctcgtcaaaatataccagattatattaacccagacgcgaaatttttgctccaaaagtatggtaaccggtctcgacacgacattttttgttaaatgcaaacgttaaagagtactgtagcttcaaaatttcagaattcacatttttattttttaaaactaccataaaacatctataacataaattctaccaaaacaaaactacagtactctttaatggcgcacacatttttggattttacacaaatttgtcgcgtcgagaccgggtaccgtatttttagcgcaaattttgtgactgggtcaatatcacgtcaatattattaataacacatcaataattaattaatactgtgggaatattggttggtgatagttgtatatcctatgcgcttgtcttattcggcatgttgtaaatattcgtcgttgtgtcatgatcatcatctcttcgatcttcatcaacaccgtcttcaacaccggatcggctgaaaccaacgaatcataaaaatgcagcaagcatgttccagGCAACAATCGAAATAATACTTGTGGTGTTGGCGTTGGTGCAGGTTCCGATTATGTTGTTTGCGAAACCATATTTTCTGTATCGCCGAGACAAGCAACAATCGAGATATAGCACTTTGACAGCAGAGTCAAATCAACATCAGgtaaacaattggtgatgggtagtttttgcatgattgtattagttttattctgcactttttccaatattattgaatcgacaccaattttataggctaatgtttttgaattcagAGTGTTCGTGCTGATATCAACCAGGATGACGCAGAAGTTGTTCACGCGCCAGAGCAAACTCCAAAACCAAGTGGTCACGGACATGGGCATGGTGATGGGCCACTTGAGATGGGGGATGTGATGGTGTACCAGGCTATTCACACAATCGAGTTTGTTCTTGGATGTGTGTCACATACTGCTTCATACCTTCGTCTTTGGGCTCTTTCATTGGCTCATGCTCgtaagtaaagaaaataatagaaaatctcaaagaagaactgatacgttaaaaagtaaaaaatttttgattgtttaaaagcctaaataataattatagaatagaaaaccctaaaattattttaccgtaaaaacgaaacaattatcgaaataaattttattttctagAGCTCTCTGATGTTCTCTGGACAATGGTTTTCCGTAATGCATTCGTTTTGGATGGATACACTGGAGCTATTGCCACTTACATTCTCTTCTTCATCTTTGGATCGTTGTCAGTGTTCATTTTGGTACTCATGGAAGGTCTTTCCGCATTCCTTCACGCTCTTCGTCTTCATTGgttcgttttctaattcaaaattagacattattaagaaaccatgagttcatgagaatgcctacttgccggcgcgaaacaagcggcagcagtgagagcatgcggcgacgagagatttaggtgccttcgctacgagatatttccgcgccaaaacggtagccattctcatgaactcatgatttcttaatacacagtcatttactgatattcaataattttcagGGTCGAGTTCCAATCAAAGTTCTATGGAGGGCTTGGATATGAGTTCGCTCCATTCTCATTCGAAAAAATCCTTGCTGAAGAGCGTGAAGCTGAAGAGAATCTCTAAgatcacctcgacgacttcaaacagtgtgacatcgacgttcgacaaatctttaattatttatttctagtagatatatacttctatttgaatattgtgtcgtgttgtgcttttttcttcttgtgtttgtgcatagagtttcccctcatcccccagccatctcctttctctaaaattgttccattttcctttcggtgaccagaatctgaattttcttcttctcgcatttttaaaattcatcttattttcttctaaattcttgcttcctgtctctatttcttttcatatttcagtctagttctcttctattgtgatgactttatgtatttcttcttaatttattccttttcttgaaagtaccgatcgctcgggatttccattttcgccaatattttgtatttcggtattgcaagctttctaatcatttagtaaatcatatttttattttaagttttttcttttcgtaaatttagtttgtctcgaattttcgattgccgatcgtcatcgccactaaccgttgaataaataagttgattgcaaacaaagtggaatcgctagctccatgacaagacagtaaatttctgaaggctatagtactattacacagacgcgaaatttggactatttttgctccaaaaatacgattccccggtctcggcacgaaaatgttttgtaattgtaaactaatgtgagccttcaaagagtacagtagcggaatgttcacaatttttggctatgtatttttttaacaattgaagcaatcaaaacatattttaacaaaaaatacgggaaaaattaaattcgcacacatttttgtctttaacgaaagattcttgcgtcaaaaatcgcaagtttttgcctcagagtaataataagctaaacattttaacccctcatcacaagtggaagcttacaaaaaataaaaattttgcagagaaatgtcaaagaaattgaagccattcgaaattttagaggattcgtgtgcatcagtatgtatttggcttaacggtgaacctacggcaatcagcaatcgcgctgaaaatttatggaataaggccaaatatcgagttgcaactgatggagctgttaatgagattcttaaaaggtgatctaggatccagaaattgaaaattatcgtaaaccgagttttggatttcagaaagagtttcgtcgaatggcctcatattatctgcggagatttcgattcaataaataaacagattgatacaaaaaatgcaaagttagtttaaattctactgaaattaaaaattaatataggcattactcaacttcattgtaatcgtgtttcatgtttgataacatcttctattaatgagcaatgatagaattactgtagggttactgtagtgatcacaaagaattattactgtagcggctgttgaatattagctaaaagaatatataggcgtgaacgttgaaaataaaattaaaatattattgagttgtgtttttaatactggaagactgaaagctacaattcgttgagaagagtgtattgaccaggtcataataatgattttgcactttttttggtatttctggcttgccaactaaatgttattcattctgtgttcaggactaaaaaaataaaaatattttgtcgaaaattgttcttaatgttgttttagagtcgtccatctgcctgatcaagactacacagatctctcgaagagcgttcagtggtgcttagagcagaaaacactaacaagctgggaattcgagaatatcgttgttctaggaggtctcaatggacgatttgatcacaccatgtcaacgttatcatctttaataagattcgttgattctcaaactcctgtgatcgttttggattctagaaatttggttctcgctgttcctacagtaatccttggcaggcctatgtgaacattaagagttaaattcagggggattcaaatcttgatgtcaatcttgaaatgacaacaaaaatgtgtggaatcattccaattgttcaaaaggagacaatcgtcagttcaattggactaaaatatgaaatgggtataatccaatgtttcaacatcatttctattaaacgtttccatttcagaaaaccttgctcttgaatttggaaaacttatcagcacgtcgaatgaagttaccacgagccaagtatttttgaaatcttcgtcgtctctgattttttcaattgaacttgaaaattgggtctacaaacttgattctctatagtatcacattttatggtccctcttaattcacaacttttcattcctttgctattcaactgttctattttctttttattccatttttcctagttttcaccggtactatataattatctacaatattataatacactttattccctgtaccattttgtgttgaaaacgaattaataaaaataaaaacgaattaatagtatgagattaaaattttcattttaaaagcaatgttatttgtttaaaaaatatccaattctaatgaattatctgcgaatatccgatagcgatttcaaaaatctaatgaaaattgaaattc'

tag = 'INSERT_THIS_SEQUENCE'

insert_end = 'C'

While we're starting by inserting the tag at the C-terminus, we want our code to work whether we're inserting the tag at the C-terminus or the N-terminus. Because of this, as you are writing your code, you'll want to switch between inserting at the C-terminus and the N-terminus  (by modifying the variable in the code cell above), ensuring that your code works in both cases. Whenever changing the insertion terminus in the first code cell above, you'll likely want to run ALL the other code cells again. You can do so by clicking `Run all` in the `Runtime` tab at the top of the page.

 ---
# Find the insert position
In order to find the PAM you want to use, you'll want to determine what region you're searching around. Find the position (i.e. the index) in the target sequence where the tag will be inserted. If you're inserting the tag at the beginning of the gene, where is the insert position relative to the start codon?  The insert position should be the index of the correct base.

![picture](https://drive.google.com/uc?export=view&id=1-tyTN4S7d61a9iQUj59q0k9_zbF_kvSD)

If you're inserting the tag at the end of the gene, where is the insert position relative to the stop codon?  The insert position should be the index of the correct base relative to the stop codon.

![picture](https://drive.google.com/uc?export=view&id=1xjlyKEEkFH3IY-mTycE7cSD-RVqvjcrV)

It should be clearer later why we're choosing our indicies this way, but see if you can figure it out for yourself (Hint: how would we grab the sequence of the target gene all the way up until the insert position?)

In the code cell below, find the index of the insert position in the target sequence. Doing so will first require you to find the position of the start or stop codon -- determined by the terminus of the insertion.

There is a hint below, if you think you need it.


In [ ]:
#@title <font color='green'>Click here for a hint</font>
%%capture

'''
We can use the wormbase format to our advantage here
'''

In [ ]:
# Find the index of the insert position

In [ ]:
#@title <font color='green'>Click here for another hint</font>
%%capture

'''
You can use 2 separate for loops - one if we are tagging the N-terminus and one if we are tagging the C-terminus
'''

In the code cell below, print the insert position you found above to ensure that your code is running properly. Remember to switch your insert terminus (the `insert_terminus` variable) to `"N"` and check that your code works in both cases. You'll need to run all of the code cells up until this point again.

In [ ]:
# Print the insert position you found above

---
# Finding the PAM

Now that we know where we're inserting our tag, we want to find the closest PAM to the insert position. We want to find the PAM where the **distance between the PAM and the insert position is minimized**.  For simplicity's sake, let's say the "position" of the PAM is the index of the first (closest to the start of the gene) base in the PAM sequence, either the N in <font color='red'>**N**</font>**GG**, or the C in <font color='red'>**C**</font>**CN**. It's important to reiterate here that the PAM _is_ strand-specific. If we find the sequence "CCN", the PAM is technically still "NGG" it's just on the reverse strand. Biologically speaking, the first base in the PAM is always the "N", we're just making a simplification here.

As such, we also want to store the strand that the PAM is on, either forward (`"+"`) or reverse (`"-"`).

There is a hint below, if you think you need it.

In [ ]:
#@title <font color='green'>Click here for a hint</font>
%%capture

'''
loop through the sequence, determining if
next three bases constitute a PAM, and
checking the distance to the insert position
(make sure your search is NOT case-sensitive)
'''

In [ ]:
# Find the PAM closest to the insert position

# Figure out which strand the PAM is on forward (+) or reverse (-)

Great! You should now have the position of the PAM (i.e. the position of the base in the PAM closest to the 5' end of the forward strand) and the strand that the PAM is on stored in variables. To ensure that your code ran correctly, in the code cell below, print:
1. The PAM strand
2. The sequence of the PAM. Don't worry about which strand the PAM is on, just print it as it appears in your target sequence above.

Double check that the output makes sense given the target gene sequence and your choice of terminus. As a reminder, you should check that the output makes sense both when inserting at the N-terminus and when inserting at the C-terminus. 

<font color="red">**BIG NOTE:**</font> For this example gene sequence, when inserting at the N-terminus, the closest PAM should be on the forward strand, and when modifying the C-terminus, the closest PAM should be on the reverse strand. This isn't always going to be the case, but it is for the sequence we've chosen here. **You will use this information throughout this week's assignment and next week's.**

In [ ]:
# Print the sequence of the PAM as it appears in the target sequence

# Print which strand the PAM is on

---
# Finding the crRNA sequence

We should now know the position of the PAM closest to the insert position, and we should know whether this PAM occurs on the forward strand or the reverse strand. Let's use this information to find our first CRISPR reagent: **the crRNA sequence**.

Recall that the crRNA sequence is identical to the 20bp directly upstream of the _PAM_. This means that the crRNA sequence depends on which strand the PAM is on. If our PAM is on the forward strand, the crRNA sequence will be the 20bp directly before NGG in our target sequence:

<center>.....GACCT<font color='green'>TGAGGCCCCGCTCGGTTTTA</font><font color='red'>NGG</font>ACTTT....</center>
<center>↓</center>
<center><font color='green'>TGAGGCCCCGCTCGGTTTTA</font></center>

But, if our PAM is on the reverse strand, the crRNA sequence will be the _reverse complement_ of the 20bp directly after CCN in our target sequence:

<center>.....AAAGT<font color='red'>CCN</font><font color='green'>TAAAACCGAGCGGGGCCTCA</font>AGGTC....</center>
<center>↓</center>
<center><font color='green'>TAAAACCGAGCGGGGCCTCA</font></center>
<center>↓</center>
<center><font color='green'>TGAGGCCCCGCTCGGTTTTA</font></center>

For this part of the assignment, we need to:
1. Grab the appropriate 20bp based on the strand that our chosen PAM falls on
2. _If_ our PAM is on the reverse strand, find the reverse complement of those 20bp.
3. Convert the DNA sequence of our crRNA to an RNA sequence

## Finding the reverse complement

Let's start by writing a function that will return the reverse complement of the sequence we give it as an argument. Remember, we want the _reverse_ complement, not just the complement. When returning the reverse complement, **make the sequence uppercase**. Do so in the code cell below.

There is a hint below, if you think you need it.

In [ ]:
#@title <font color='green'>Click here for a hint</font>
%%capture

'''
Try using a dictionary like this to find the
complement of a single base:

{'A':'T', 'C':'G', 'G':'C', 'T':'A'}

Go through the input sequence one base at a time
'''

In [ ]:
# Write a function to find the reverse complement of an input sequence

In the code cell below, double check that your function works by running it on the test sequence provided, and printing the output.

In [ ]:
test_seq = 'TCAATTAATAAGGGGGTAGA'

# Check that your function works on this sequence

## Converting from DNA to RNA

We've done the hard part, now let's write a function that will take a DNA sequence as an argument and will return the corresponding RNA sequence as an output (just convert the T's to U's, no need to take the reverse complement). As before, when returning the RNA sequence, **make the sequence uppercase**. Do so in the code cell below.

In [ ]:
# Write a function to convert DNA to RNA

As above, in the code cell below, check that your function works by running it on the test sequence provided, and printing the output.

In [ ]:
test_seq = 'TCAATTAATAAGGGGGTAGA'

# Check that your function works on this sequence

## Putting it together

Now that you have one function that can find the reverse complement of a DNA sequence, and another that can convert from DNA to RNA, let's put it together to find the sequence of our crRNA for this experiment. Use the position and strand of the PAM to find the correct 20 bp, use the reverse complement function if necessary, and finally convert from DNA to RNA. Print the final crRNA sequence **as RNA**. Do so in the code cell below.

Remember to make sure your code works for both the C-terminus insertion and the N-terminus insertion.

In [ ]:
# Use the location and strand of the PAM to find the crRNA sequence

# Print the crRNA sequence (in RNA) that you found